In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
def convert_xlsx_to_csv(input_file, output_file):
    # Read the XLSX file into a pandas DataFrame
    df = pd.read_excel(input_file)

    # Write the DataFrame to a CSV file
    df.to_csv(output_file, index=False)


input_file = 'Online Retail.xlsx'  
output_file = 'output.csv'  

convert_xlsx_to_csv(input_file, output_file)

In [4]:
# Read the dataset into a DataFrame
df = pd.read_csv(output_file)  
# df.head()

In [5]:
print(df.describe())
print(df.isna().sum())

            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13953.000000
50%         3.000000       2.080000   15152.000000
75%        10.000000       4.130000   16791.000000
max     80995.000000   38970.000000   18287.000000
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [6]:
# -------------------- Data Preprocessing --------------------

# removing negative values
df_filtered = df[(df['Quantity'] >= 0) & (df['Quantity'] %1 ==0)]
df_filtered = df_filtered.reset_index(drop=True)

# df["Quantity"] = df["Quantity"].astype(int)
df_filtered["InvoiceDate"] = pd.to_datetime(df_filtered["InvoiceDate"])
df_filtered["UnitPrice"] = df_filtered["UnitPrice"].astype(float)
# df["CustomerID"] = df["CustomerID"].astype(int)

df_filtered.dropna(subset=["CustomerID"], inplace=True)  # Drop rows with missing CustomerID values
df_filtered["Description"].fillna("Unknown", inplace=True)  # Fill missing Description values with "Unknown"

df_filtered["Description"] = df_filtered["Description"].str.strip()
df_filtered["Country"] = df_filtered["Country"].str.strip()

df_filtered.drop_duplicates(inplace=True)
df_filtered.reset_index(drop=True, inplace=True)



In [7]:
# check again if the data is cleaned or not

# df_filtered.describe()
df_filtered.isna().sum()


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [8]:
# --------- Feature engineering --------------

df_filtered['Date'] = pd.to_datetime(df_filtered['InvoiceDate'])
df_filtered['Day'] = df_filtered['InvoiceDate'].dt.day
df_filtered['Month'] = df_filtered['InvoiceDate'].dt.month
df_filtered['Year'] = df_filtered['InvoiceDate'].dt.year

In [13]:
# -----------Model Selecetion and training--------------

model = LinearRegression()

X = df_filtered[['CustomerID', 'Day', 'Month', 'Year']]
y = df_filtered['Quantity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

# -------Model evaluation------
y_pred = model.predict(X_test)
## Uncomment this to see the mean absolute error --
# mse = mean_squared_error(y_test, y_pred)
# print(f"Mean Squared Error: {mse}")

# ----------Prediction -------------
customer_id = int(input("Enter CustomerID: "))
date_input = input("Enter Date: ")
date = pd.to_datetime(date_input)
new_data = pd.DataFrame({'CustomerID': [customer_id], 'Day': [date.day], 'Month': [date.month], 'Year': [date.year]})
quantity_pred = model.predict(new_data)
print(f"Predicted Quantity: {quantity_pred[0]}")

Predicted Quantity: 13.928936609905804
